## Setup

In [19]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

## Pipe the dataset to a model

In [20]:
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_train_preprocessed = X_train / 128
X_train_preprocessed -= 1
print("X_train_preprocessed:", X_train_preprocessed)
print("X_train_preprocessed.shape:", X_train_preprocessed.shape)
print("y_train:", y_train)
print("y_train.shape:", y_train.shape)

X_train_preprocessed: [[[[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   ...
   [ 0.2578125  0.1875     0.2109375]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  [[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   ...
   [ 0.2578125  0.1875     0.2109375]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  [[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.25       0.203125   0.21875  ]
   ...
   [ 0.2578125  0.1875     0.1953125]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  ...

  [[-0.625     -0.6171875 -0.5859375]
   [-0.65625   -0.6484375 -0.6171875]
   [-0.6640625 -0.65625   -0.6171875]
   ...
   [ 0.4609375  0.4140625  0.4296875]
   [ 0.4765625  0.40625    0.4296875]
   [ 0.46875    0.3984375  0.40625  ]]

  [[-0.625     -0.63281

In [21]:
label_names = ['undamaged', 'damaged']

In [22]:
inception_v3 = tf.keras.applications.InceptionV3(input_shape=(224, 224, 3), include_top=False)
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(inception_v3.layers):
    print(i, layer.name)
# we will freeze the first 249 layers and unfreeze the rest:
for layer in inception_v3.layers[:249]:
    layer.trainable = False
for layer in inception_v3.layers[249:]:
    layer.trainable = True

0 input_6
1 conv2d_470
2 batch_normalization_470
3 activation_470
4 conv2d_471
5 batch_normalization_471
6 activation_471
7 conv2d_472
8 batch_normalization_472
9 activation_472
10 max_pooling2d_20
11 conv2d_473
12 batch_normalization_473
13 activation_473
14 conv2d_474
15 batch_normalization_474
16 activation_474
17 max_pooling2d_21
18 conv2d_478
19 batch_normalization_478
20 activation_478
21 conv2d_476
22 conv2d_479
23 batch_normalization_476
24 batch_normalization_479
25 activation_476
26 activation_479
27 average_pooling2d_45
28 conv2d_475
29 conv2d_477
30 conv2d_480
31 conv2d_481
32 batch_normalization_475
33 batch_normalization_477
34 batch_normalization_480
35 batch_normalization_481
36 activation_475
37 activation_477
38 activation_480
39 activation_481
40 mixed0
41 conv2d_485
42 batch_normalization_485
43 activation_485
44 conv2d_483
45 conv2d_486
46 batch_normalization_483
47 batch_normalization_486
48 activation_483
49 activation_486
50 average_pooling2d_46
51 conv2d_482
52

In [ ]:
model = tf.keras.Sequential([
  inception_v3,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(label_names), activation="softmax")])

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [ ]:
len(model.trainable_variables)

38

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 11,118,978
Non-trainable params: 10,687,904
_________________________________________________________________


In [ ]:
model.fit(x=X_train_preprocessed, y=y_train, batch_size=100, epochs=100, validation_split=0.1)

Epoch 1/100
10/54 [====>.........................] - ETA: 14:11 - loss: 1.0221 - acc: 0.6520